### Topic Modeling: Demand for Cybersecurity Professionals  

In [3]:
import pandas as pd 
import numpy as np
import nltk 
from nltk.corpus import stopwords 
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### Data Preprocessing 
Indeed Data 

In [4]:
df = pd.read_csv("C:/Users/hanna/Scape-Save-DAEN690/Datasets/indeed_cleaned.csv.gz")

In [5]:
#Remove duplicate postings if the job title, description and location are the same 
df.drop_duplicates(subset=['job_title', 'description_text', 'location'], inplace=True)

In [6]:
#Set to lower case so capitalized titles don't get counted separately
df['job_title'] = df['job_title'].apply(lambda x: str(x).lower())
#Set to lower case for job descriptions 
df['description_text'] = df['description_text'].apply(lambda x: str(x).lower())
#remove numerical values from job description
df['description_text'] = df['description_text'].str.replace('\d+', '')
#remove links from job description 
df['description_text'] = df['description_text'].str.replace('http\S+|www.\S+', '', case=False)



In [7]:
df

,company_name,country,country_code,current_url,date_posted_parsed,description,description_text,domain,job_title,job_type,jobid,location,region,timestamp,qualifications,Unnamed: 15
0,NaN,US,UM,https://www.indeed.com/viewjob?jk=00cf07391044...,NaN,<div>\n <p>DomainTools is looking for a talent...,domaintools is looking for a talented senior a...,www.indeed.com,senior cyber security account executive (remot...,"[""Full-time""]",00cf07391044c067,Remote,NaN,12/24/2022,NaN,NaN
1,BTS Software Solutions,US,NaN,https://www.indeed.com/viewjob?dae2824b04=b80b...,NaN,<p><b>Cyber Defense Analyst III</b><br><b>REQ ...,cyber defense analyst iiireq id: -just awarded...,www.indeed.com,cyber defense analyst iii,"[""Full-time""]",395221d50cb09389,"San Antonio, TX 78251",NaN,12/24/2022,NaN,NaN
2,Change Healthcare,US,US,https://www.indeed.com/viewjob?jk=434af1ddd634...,NaN,<div>\n <p><b>Title</b><br> Senior Cybersecuri...,title senior cybersecurity incident response a...,NaN,senior cybersecurity incident response analyst,"[""Full-time""]",434af1ddd63444bf,South Carolina,NaN,07-03-2022,NaN,NaN
3,NaN,US,UM,https://www.indeed.com/viewjob?jk=1547fa2c6b50...,NaN,<div>\n <p><b>About the Team</b></p> \n <p> Jo...,about the team \n joining capco means joining...,www.indeed.com,managing principal - cybersecurity,NaN,1547fa2c6b504d38,Remote,NaN,12/24/2022,NaN,NaN
4,HALVIK,US,US,https://www.indeed.com/viewjob?jk=4770813092f5...,NaN,<div>\n <div>\n <div>\n If you are looking ...,if you are looking for a rewarding career with...,www.indeed.com,cybersecurity systems analyst,"[""Full-time""]",4770813092f57d23,"Tampa, FL",NaN,12/24/2022,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11877,Lighthouse Lab Services,US,US,https://www.indeed.com/viewjob?jk=c43bf16c68ee...,NaN,<div>\n <p>Lighthouse Lab Services is excited ...,lighthouse lab services is excited to be worki...,indeed.com,director of it management (ciso),"[""Full-time""]",c43bf16c68ee5f6a,"Atlanta, GA",NaN,7/20/2022,NaN,NaN
11879,NaN,US,US,https://www.indeed.com/viewjob?jk=8c317805cd78...,NaN,<div>\n Note: By applying to this position you...,note: by applying to this position you will ha...,NaN,"program manager, compliance, google cloud ciso","[""Full-time""]",8c317805cd7856c3,"Raleigh, NC",NaN,07-03-2022,NaN,NaN
11880,Abnormal Security,US,US,https://www.indeed.com/viewjob?jk=d55a050f564b...,2022-06-25T10:31:20.963Z,"<div>\n <h2 class=""jobSectionHeader""><b>About ...",about abnormal security \n users and companie...,NaN,head of program management - office of the ciso,NaN,d55a050f564b9d96,"Washington, DC",NaN,07-03-2022,NaN,NaN
11881,Google,US,US,https://www.indeed.com/viewjob?jk=1f951e7bd07c...,NaN,<div>\n Note: By applying to this position you...,note: by applying to this position you will ha...,NaN,"lead technical program manager, compliance ena...","[""Full-time""]",1f951e7bd07c6a1b,"Raleigh, NC",NaN,07-03-2022,NaN,NaN


In [8]:
#Create function to categorize role types 

def role_type(x):
    if 'lead' in x:
        return 'senior'
    elif 'senior' in x:
        return 'senior'
    elif 'sr' in x:
        return 'senior'
    elif 'chief' in x:
        return 'upper mgmt'
    elif 'manager' in x:
        return 'upper mgmt'
    elif 'principal' in x:
        return 'upper mgmt'
    elif 'junior' in x:
        return 'entry-level'
    elif 'jr' in x:
        return 'entry-level'
    elif 'entry-level' in x:
        return 'entry-level'
    elif 'entry level' in x:
        return 'entry-level'
    elif 'professor' in x: 
        return 'academia'
    else:
        return 'other'
    
df['role_type'] = df['job_title'].apply(role_type)

#Can be used for visualizations 
job_df = df.groupby('role_type').count().reset_index()[['role_type', 'job_title']]


In [9]:
#Comparing job descriptions using cosine similarity will help remove job descriptions that are almost identical with additional word difference 
#Helps reduce processing time of the model 

corpus= df['description_text']

count_vect = CountVectorizer()

x_counts = count_vect.fit_transform(corpus)

cos_df = pd.DataFrame(cosine_similarity(x_counts))
 
i, j = np.indices(cos_df.shape).reshape(2, -1)

cos_values = cos_df.values.reshape(-1)

cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'score':cos_values})

#similarity score >0.99 indicates that the job descriptions are very similar with slight variations in wordings
cos_rem = cos_sim_df[(cos_sim_df['score']>0.99)&(i!=j)]

cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)

#Drop rows with >0.99 similarity score from original dataframe
df = df[~df.index.isin(drop_rows)]
df = df.reset_index()
df.drop(columns={'index'}, inplace=True)

CareerOneStop Data

In [10]:
cos_df = pd.read_excel("C:/Users/hanna/Scape-Save-DAEN690/Datasets/careeronestop_data.xlsx") 

In [11]:
cos_df.drop_duplicates(subset=['job_title', 'description', 'location'], inplace=True)
#Set to lower case so capitalized titles don't get counted separately
cos_df['job_title'] = cos_df['job_title'].apply(lambda x: str(x).lower())

In [12]:
#Set to lower case so capitalized titles don't get counted separately
cos_df['job_title'] = cos_df['job_title'].apply(lambda x: x.lower())
#Set to lower case for job descriptions 
cos_df['description'] = cos_df['description'].apply(lambda x: str(x).lower())
#remove numerical values from job description
cos_df['description'] = cos_df['description'].str.replace('\d+', '')
#remove links from job description 
cos_df['description'] = cos_df['description'].str.replace('http\S+|www.\S+', '', case=False)

In [13]:
#apply function to add job roles 
cos_df['role_type'] = cos_df['job_title'].apply(role_type)

job_df = cos_df.groupby('role_type').count().reset_index()[['role_type', 'job_title']]

In [14]:
cos_df.shape

(8920, 8)

In [15]:
corpus= cos_df['description']

count_vect = CountVectorizer()

x_count_cos = count_vect.fit_transform(corpus.values.astype('U'))

cosi_df = pd.DataFrame(cosine_similarity(x_count_cos))

i, j = np.indices(cosi_df.shape).reshape(2, -1)

cos_values = cosi_df.values.reshape(-1)

cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'score':cos_values})

#similarity score >0.99 indicates that the job descriptions are very similar with slight variations in wordings 
cos_rem = cos_sim_df[(cos_sim_df['score']>0.99)&(i!=j)]

cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)

cos_df = cos_df[~cos_df.index.isin(drop_rows)]
cos_df = cos_df.reset_index()
cos_df.drop(columns={'index'}, inplace=True)

#### BERTopic Algorithm

In [16]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic 
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#cluster_model = KMeans(n_clusters=10)

# Step 4 - Tokenize topics
# ngram_range set to 2 so the topics extracted can also be 'two-words' instead of one 
vectorizer_model = CountVectorizer(ngram_range=(1,2), stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - Fine-tune topic representations with 
# reduces redundacy and improve diversity of keywords using MMR 
representation_model = MaximalMarginalRelevance(0.4)

topic_model = BERTopic(
  embedding_model=embedding_model, # Step 1: Embedding        
  umap_model=umap_model, # Step 2: Dimensionality reduction                   
  hdbscan_model=hdbscan_model, # Step 3: Clustering              
  vectorizer_model=vectorizer_model, # Step 4: Tokenizer         
  ctfidf_model=ctfidf_model, # Step 5: Weighting scheme                
  representation_model=representation_model) # Step 6: Fine-tune topic representation 

topic_model_in = BERTopic(
  embedding_model=embedding_model,          
  umap_model=umap_model,                    
  hdbscan_model=hdbscan_model,              
  vectorizer_model=vectorizer_model,       
  ctfidf_model=ctfidf_model,                
  representation_model=representation_model) 

In [26]:
#CareerOneStop model
topic, prob = topic_model.fit_transform(cos_df['description'].values.astype('U'))

In [27]:
#Indeed Model
topic_in, prob_in = topic_model_in.fit_transform(df['description_text']) 


In [19]:
#before topic reduction: model contains 242 topics 
topic_model.get_topic_freq()

,Topic,Count
0,-1,921
1,0,189
2,1,76
3,2,60
4,3,57
...,...,...
242,241,5
243,242,5
244,243,5
245,244,5


In [20]:
#topic reduction model, reduce topic count to 50 

topic_model.reduce_topics(cos_df['description'], nr_topics=50)
topics = topic_model.topics_

In [30]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,924,-1_security_experience_cybersecurity_management
1,0,1568,0_experience_management_cybersecurity_requirem...
2,1,726,1_experience_information security_deloitte_tec...
3,2,250,2_cloud_experience_architect_azure
4,3,199,3_analyst_ttec_soc analyst_competitive intelli...
5,4,191,4_rubrik_minimum maximum_salaries role_salaries
6,5,103,5_xerox_cyber_incident response_business teams
7,6,90,6_northrop grumman_security_dod_clearance
8,7,72,7_bcg_cybersecurity_mobile_tda
9,8,59,8_nrel_security_experience_fusion technology


In [28]:
#model with careeronestop data
topic_model.visualize_barchart(top_n_topics=5)

In [29]:
#model with indeed data 
topic_model_in.visualize_barchart(top_n_topics=5)

In [23]:
freq = topic_model_in.get_topic_info() 
print(freq)

     Topic  Count                                               Name
0       -1   2306        -1_experience_management_cybersecurity_risk
1        0     95             0_students_teaching_faculty_curriculum
2        1     89                 1_automotive_zf_uber_cybersecurity
3        2     88  2_security officer_dod_officer isso_systems se...
4        3     58  3_gsk_security architecture_security architect...
..     ...    ...                                                ...
440    439      5  439_business analyst_kawartha_cyber security_p...
441    440      5  440_cibc_cibc information_strengths ambitions_...
442    441      5      441_chase_jpmorgan chase_risk control_banking
443    442      5       442_new york_chamber_alpha omega_mvp systems
444    443      5  443_cloud forensics_forensics_forensics analys...

[445 rows x 3 columns]


In [24]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,921,-1_security_experience_cybersecurity_risk
1,0,2007,0_experience_management_cybersecurity_informat...
2,1,220,1_ctg_cerner millennium_application_millennium...
3,2,204,2_cloud_azure_architect_services
4,3,175,3_rubrik_minimum maximum_salaries role_cloud
5,4,152,4_competitive intelligence_relations_analyst r...
6,5,120,5_gdit_requirements_cybersecurity_years
7,6,100,6_applicable experience_years applicable_wells...
8,7,78,7_security_network environment_network securit...
9,8,68,8_cardinal_cardinal health_organization_health...


In [25]:
#keywords present in our top topics 
topic_model.get_topic(freq.iloc[15]["Topic"])

[('click applicants', 0.02739537698842505),
 ('residing brazil', 0.019803918514052617),
 ('financial', 0.018384542937388153),
 ('applicants', 0.018122333504819437),
 ('accessibility support', 0.015507425807820304),
 ('base salary', 0.013785058106428915),
 ('security solutions', 0.011310817124772988),
 ('support tiaa', 0.010953477711034812),
 ('applicants tiaa', 0.010953477711034812),
 ('nuveen affiliates', 0.010953477711034812)]